# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [64]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [65]:


city_weather_df = pd.read_csv("weather_data.csv")
city_weather_df


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Albany,31,US,1601021058,80,42.60,-73.97,61.00,2.21
1,1,Dikson,100,RU,1601021059,86,73.51,80.55,44.49,30.36
2,2,Kapaa,90,US,1601020979,74,22.08,-159.32,80.60,16.11
3,3,Baní,19,DO,1601021062,82,18.28,-70.33,80.01,3.00
4,4,Barrow,90,US,1601021063,97,71.29,-156.79,32.00,11.41
...,...,...,...,...,...,...,...,...,...,...
560,560,Paamiut,100,GL,1601021661,74,61.99,-49.67,43.45,42.26
561,561,Kokkola,46,FI,1601021789,76,63.84,23.13,57.20,14.99
562,562,Broome,1,US,1601021791,87,42.25,-75.83,57.20,2.42
563,563,Nishihara,75,JP,1601021792,74,26.18,127.76,82.40,10.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [66]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [67]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_weather_df[["Lat", "Lng"]].astype(float)

# Create a poverty Heatmap layer
humidity_fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=city_weather_df['Humidity'], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

humidity_fig.add_layer(heat_layer)
humidity_fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [68]:
ideal_cities_df = city_weather_df.loc[(city_weather_df["Max Temp"] > 70) & (city_weather_df["Max Temp"] < 80) & (city_weather_df["Cloudiness"] == 0) & (city_weather_df["Humidity"] < 50), :]
ideal_cities_df = ideal_cities_df.dropna(how='any')
del ideal_cities_df['Unnamed: 0']
hotel_df=ideal_cities_df.reset_index(drop=True)
hotel_df


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Yumen,0,CN,1601021218,14,40.28,97.20,79.36,14.09
1,Beyneu,0,KZ,1601021374,26,45.32,55.20,75.72,15.99
2,Sayyan,0,YE,1601021472,20,15.17,44.32,72.01,14.61
3,Jaciara,0,BR,1601021478,47,-15.97,-54.97,75.27,1.12
4,Aksu,0,CN,1601021488,16,41.12,80.26,79.95,4.43
5,Abhā,0,SA,1601021606,10,18.22,42.51,78.80,11.41
6,Gazojak,0,TM,1601021607,18,41.19,61.40,78.35,10.00
7,Xiaoshi,0,CN,1601021729,47,41.30,124.12,75.20,4.47


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [69]:
hotel_df = ideal_cities_df.copy()
hotel_df["Hotel Name"] = ""
hotel_df.reset_index(inplace=True,drop=True)
hotel_df


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Yumen,0,CN,1601021218,14,40.28,97.20,79.36,14.09,
1,Beyneu,0,KZ,1601021374,26,45.32,55.20,75.72,15.99,
2,Sayyan,0,YE,1601021472,20,15.17,44.32,72.01,14.61,
3,Jaciara,0,BR,1601021478,47,-15.97,-54.97,75.27,1.12,
4,Aksu,0,CN,1601021488,16,41.12,80.26,79.95,4.43,
5,Abhā,0,SA,1601021606,10,18.22,42.51,78.80,11.41,
6,Gazojak,0,TM,1601021607,18,41.19,61.40,78.35,10.00,
7,Xiaoshi,0,CN,1601021729,47,41.30,124.12,75.20,4.47,


In [70]:
#
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get latitude & longitude
    lat = row["Lat"]
    long = row["Lng"]

    #loop through parameters
    params["location"] = f"{lat},{long}"
    #loop through this url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #make API request
    print(f"Retrieving Results: {row['City']}.")
    response = requests.get(base_url, params=params).json()
   
    
    #results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Retrieving Results: Yumen.
Missing field/result... skipping.
Retrieving Results: Beyneu.
Closest hotel is NUR.
Retrieving Results: Sayyan.
Closest hotel is الاستاذة مروى اليوسفي.
Retrieving Results: Jaciara.
Closest hotel is Taba Hotel.
Retrieving Results: Aksu.
Closest hotel is Pudong Holiday Hotel.
Retrieving Results: Abhā.
Closest hotel is Abha Hotel.
Retrieving Results: Gazojak.
Missing field/result... skipping.
Retrieving Results: Xiaoshi.
Closest hotel is Junsheng Fengweilou Farmstay.


In [62]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [63]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Yumen,0,CN,1601021218,14,40.28,97.20,79.36,14.09,
1,Beyneu,0,KZ,1601021374,26,45.32,55.20,75.72,15.99,NUR
2,Sayyan,0,YE,1601021472,20,15.17,44.32,72.01,14.61,الاستاذة مروى اليوسفي
3,Jaciara,0,BR,1601021478,47,-15.97,-54.97,75.27,1.12,Taba Hotel
4,Aksu,0,CN,1601021488,16,41.12,80.26,79.95,4.43,Pudong Holiday Hotel
5,Abhā,0,SA,1601021606,10,18.22,42.51,78.80,11.41,Abha Hotel
6,Gazojak,0,TM,1601021607,18,41.19,61.40,78.35,10.00,
7,Xiaoshi,0,CN,1601021729,47,41.30,124.12,75.20,4.47,Junsheng Fengweilou Farmstay
